In [1]:
import time
import json
import pickle
import numpy as np
import pandas as pd
from classes import *
from datetime import datetime
import plotly.graph_objects as go

In [2]:
with open("lpf_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

In [3]:
def plot_data_arr(sensor_data, mat, sensor, labels, skip=0, cut=0):
    title = f"Matrix {mat}, Sensor {sensor}"
    heaters = sensor_data[f"mat_{mat}"][sensor]
    fig = go.Figure()

    for i, data in heaters.items():
        fig.add_trace(go.Scatter(x=data["Time Since PowerOn"][skip:-cut],
                                 y=data["Filtered"][skip:-cut],
                                 mode="lines",
                                 showlegend=True,
                                 name=f"H.S. {i}"))
    
    for label in labels[f"mat_{mat}"]:
        fig.add_vrect(x0=label["start"],
                      x1=label["end"],
                      annotation_text=classes[label["label"]],
                      annotation_position="top left",
                      fillcolor=colors[label["label"]],
                      opacity=0.4,
                      line_width=0)
    fig.update_yaxes(title_text="Resistance (Ohms)",
                     type="log", tickformat=".0e")
    fig.update_xaxes(title_text="Time since power on (ms)", tickformat=".0e")
    fig.update_layout(height=400, width=700,
                      title_x=0.5,
                      font_family="Times New Roman",
                      title_font_family="Times New Roman",
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=50, r=10, b=0, l=0))

    fig.update_yaxes(type="log")
    fig.show()

In [4]:
def get_df(matrix):
    with open(f"raw_data/matrix_{matrix}/{matrix}.bmerawdata") as f:
        j = json.load(f)

    column_names = [el["name"] for el in j["rawDataBody"]["dataColumns"]]
    df = pd.DataFrame(j["rawDataBody"]["dataBlock"], columns=column_names)
    return df

In [5]:
def get_tspo_val(date_str, df):
    ts = time.mktime(datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S").timetuple())
    ts = int(ts)
    values = df[df["Real time clock"] == ts]["Time Since PowerOn"].values
    return values[0]

Matrix 0 -> oil; 1 -> control

In [9]:
df = get_df(0)
print(get_tspo_val("2025-02-27 10:00:00", df))
print(get_tspo_val("2025-02-27 17:05:00", df))
df = get_df(1)
print(get_tspo_val("2025-02-27 10:00:00", df))
print(get_tspo_val("2025-02-27 17:05:00", df))

59023709
84520753
58988055
84485038


In [6]:
label_datetimes = {
    "mat_0": [
        {"start": "2025-02-27 10:30:00",
         "end": "2025-02-27 10:53:00",
         "cls": OLIVE_100},
        {"start": "2025-02-27 11:30:00",
         "end": "2025-02-27 11:53:00",
         "cls": OLIVE_95},
        {"start": "2025-02-27 12:30:00",
         "end": "2025-02-27 12:53:00",
         "cls": OLIVE_90},
        {"start": "2025-02-27 13:30:00",
         "end": "2025-02-27 13:53:00",
         "cls": OLIVE_80},
        {"start": "2025-02-27 14:30:00",
         "end": "2025-02-27 14:53:00",
         "cls": OLIVE_70},
        {"start": "2025-02-27 15:30:00",
         "end": "2025-02-27 15:53:00",
         "cls": OLIVE_50},
        {"start": "2025-02-27 16:30:00",
         "end": "2025-02-27 16:53:00",
         "cls": OLIVE_0}
    ],
    "mat_1": [
        {"start": "2025-02-27 10:30:00",
         "end": "2025-02-27 10:53:00",
         "cls": AIR},
        {"start": "2025-02-27 11:30:00",
         "end": "2025-02-27 11:53:00",
         "cls": AIR},
        {"start": "2025-02-27 12:30:00",
         "end": "2025-02-27 12:53:00",
         "cls": AIR},
        {"start": "2025-02-27 13:30:00",
         "end": "2025-02-27 13:53:00",
         "cls": AIR},
        {"start": "2025-02-27 14:30:00",
         "end": "2025-02-27 14:53:00",
         "cls": AIR},
        {"start": "2025-02-27 15:30:00",
         "end": "2025-02-27 15:53:00",
         "cls": AIR},
        {"start": "2025-02-27 16:30:00",
         "end": "2025-02-27 16:53:00",
         "cls": AIR}
    ]
}

sensor_labels = {"mat_0": [], "mat_1": []}
for matrix in range(2):
    df = get_df(matrix)
    for label_time in label_datetimes[f"mat_{matrix}"]:
        start_ts = get_tspo_val(label_time["start"], df)
        end_ts = get_tspo_val(label_time["end"], df)
        cls = label_time["cls"]
        reg_target = regression_targets[cls] if cls in regression_targets else None
        
        sensor_labels[f"mat_{matrix}"].append({
            "start": start_ts,
            "end": end_ts,
            "label": cls,
            "target": reg_target
        })

In [15]:
plot_data_arr(sensor_data, 0, 0, sensor_labels, skip=3000, cut=10)

In [16]:
plot_data_arr(sensor_data, 1, 0, sensor_labels, skip=3000, cut=10)

In [11]:
with open("sensor_labels.pkl", "wb") as f:
    pickle.dump(sensor_labels, f)

In [59]:
def plot_matrices_data_arr(sensor_data, sensor, labels, skip=0, cut=0):
    title = f"Sensor {sensor}"
    mat = 0
    heaters = sensor_data[f"mat_{mat}"][sensor]
    fig = go.Figure()

    for i, data in heaters.items():
        fig.add_trace(go.Scatter(x=data["Time Since PowerOn"][skip:-cut],
                                 y=np.log(data["Filtered"][skip:-cut]),
                                 mode="lines",
                                 showlegend=True,
                                 name=f"0 H.S. {i}",
                                 line_color="#388eff"))
    
    for label in labels[f"mat_{mat}"]:
        fig.add_vrect(x0=label["start"],
                      x1=label["end"],
                      annotation_text=classes[label["label"]],
                      annotation_position="top left",
                      fillcolor=colors[label["label"]],
                      opacity=0.4,
                      line_width=0)
    mat = 1    
    heaters = sensor_data[f"mat_{mat}"][sensor]
    for i, data in heaters.items():
        fig.add_trace(go.Scatter(x=data["Time Since PowerOn"][skip:-cut],
                                 y=np.log(data["Filtered"][skip:-cut]),
                                 mode="lines",
                                 showlegend=True,
                                 name=f"1 H.S. {i}",
                                 line_color="#ff4538"))
        
    fig.update_yaxes(title_text="Resistance (Ohms)")
    fig.update_xaxes(title_text="Time since power on (ms)", tickformat=".0e")
    fig.update_layout(height=700, width=1200,
                      title_x=0.5,
                      font_family="Times New Roman",
                      title_font_family="Times New Roman",
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=50, r=10, b=0, l=0))

    fig.show()

In [70]:
plot_matrices_data_arr(sensor_data, 0, sensor_labels, skip=2900, cut=10)